Все ДЗ состоит из двух частей. Первая часть более развернутая и экспериментальная - в ней вам нужно будет обучить архитектуру для задачи Image Classification. Вам нужно реализовать архитектуру, обучить ее, а так же выполнить некоторые дополнительные задания. Вторая часть более простая - вам просто требуется разобраться во фреймворке и запустить его на данных.

In [1]:
#libs

import torch
from torch import nn
import torch.nn.functional as F

# ДЗ 2.1 Image classification. 16 баллов.

В качестве датасета будет использовать вот этот - https://www.kaggle.com/datasets/slothkong/10-monkey-species

# Dataset

In [1]:
##TODO##

# Dataloader

In [2]:
##TODO##

## ResNet50.

Реализуйте ResNet50. Чтобы вспомнить архитектуру, обратитесь к лекции + оригинальной статье авторов+google.

Если хотите реализовать что-то поинтереснее (ViT) - делайте, будет учтено.

Что будет оцениваться в задании:
* ResNet50 from scratch **6 баллов**
* добавьте аугментаций, напишите краткое саммари почему выбрали именно такой набор аугментаций. **1 балл**
* дописать логику обучения модели **2 балла**
* дописать логику тестирования модели **1 балл**
* возьмите еще 2 модели из зоопарка моделей [torchvision](https://pytorch.org/vision/stable/models.html?), зафайнтюньте на наш датасет. Сравните их между собой и вашей реализацией. **4 балла**
* обязательно в конце напишите общий отчет о проделанной работе. Для лучшей модели проведите анализ метрик и ошибок, постройте confusion_matrix и classification report **2 балла**
* графики обучения обязательно должны присутсвовать

**NOTE**: рядом лежит ноутбук с baseline solution, вы можете отталкиваться от него, если хотите.

In [ ]:
def conv_3x3(in_channels, out_channels, stride=1, dilation=1, groups=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        dilation=dilation,
        groups=groups,
        bias=False,
    )


def conv_1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(
        in_channels, out_channels, kernel_size=1, stride=stride, bias=False
    )

In [ ]:
class BottleNeck(nn.Module):
    def __init__(
        self, in_channels, channels, stride=1, groups=1, dilation=1, downsample=None
    ):
        super(BottleNeck, self).__init__()
        norm_layer = nn.BatchNorm2d
        width = int(channels * groups)

        self.conv1 = conv_1x1(in_channels, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv_3x3(
            width, width, stride=stride, groups=groups, dilation=dilation
        )
        self.bn2 = norm_layer(width)
        self.conv3 = conv_1x1(width, channels * 4)
        self.bn3 = norm_layer(channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        skip_connection = x
        if self.downsample is not None:
            skip_connection = self.downsample(x)

        output = self.conv1(x)
        output = self.bn1(output)
        output = self.relu(output)
        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu(output)
        output = self.conv3(output)
        output = self.bn3(output)
        output += skip_connection
        output = nn.relu(output)

        return output

In [ ]:
class ResNet50(nn.Module):
    def __init__(self, num_classes):

        super(ResNet50, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=3)

        self.layer_1 = self._build_layer(channels=64, num_blocks=3)
        self.layer_2 = self._build_layer(channels=128, num_blocks=4, stride=2)
        self.layer_3 = self._build_layer(channels=256, num_blocks=6, stride=2)
        self.layer_4 = self._build_layer(channels=512, num_blocks=3, stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, num_classes)

        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.kaiming_normal_(
                    module.weight, mode="fan_out", nonlinearity="relu"
                )
            elif isinstance(module, nn.BatchNorm2d):
                nn.init.constant_(module.weight, 1)
                nn.init.constant_(module.bias, 0)

    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.max_pool(out)

        out = self.layer_1(out)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)

        out = self.avg_pool(out)
        out = torch.flatten(x, 1)
        out = self.fc(out)

        return out

    def _build_layer(self, channels, num_blocks, stride=1):

        downsample = nn.Sequential(
            conv_1x1(self.in_channels, channels * 4, stride),
            nn.BatchNorm2d(channels * 4),
        )
        blocks = []
        blocks.append(
            BottleNeck(self.in_channels, channels, stride=stride, downsample=downsample)
        )
        self.in_channels = channels * 4
        for _ in range(1, num_blocks):
            blocks.append(BottleNeck(self.in_channels, channels, stride=stride))

        return nn.Sequential(*blocks)

# Train

# Test and metrics

# Summary

# ДЗ 2.2. Object detection. 8 баллов.

Здесь я прошу вас воспользоваться популярным фреймворком для задачи object detection - YOLOv8. Но, если хотите, можете поиграться с моделями из mmdetection например, или запустить детекцию на DETR. Тоже будет оценено. YOLOv8 выбран как самый простой и удобный вариант. Ссылка на гитхаб и документацию - https://github.com/ultralytics/ultralytics.

Вам требуется поставить фреймворк, обучить модель детекции на данных, вывести графики лоссов и метрики на тестовом датасете. Покажите предсказания модели на 10 примерах из тестового набора данных. В этом задании не будет оцениваться каждый пункт по отдельности, а только вся проведенная работа в целом.

Обучать лучше на GPU, но на CPU тоже возможно. Маленькая версия YOLO с ресайзом до 320 у меня обучалась около 40 минут на CPU. Данные нужно скачать отсюда - https://public.roboflow.com/object-detection/oxford-pets/1/download/yolov8 (скачайте в формате YOLO). Имейте ввиду, что вам скорее всего нужно будет поменять пути в `data.yaml`, поставив абсолютные пути до папок. Например:

`train: /Users/mark/Desktop/Update820/train` и т.д.